# Chapter 5. Estimating Counts

[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap05.html)


In this chapter, we’ll work on problems related to counting, or estimating the size of a population.

A reminder of Bayes’s Theorem:

$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$

or

$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import Distributions as Dsts

In [ ]:
include("pmf.jl")
import .ProbabilityMassFunction as Pmf

## The Train Problem

I found the train problem in Frederick Mosteller’s, [Fifty Challenging Problems
in Probability with
Solutions](https://store.doverpublications.com/0486653552.html):

> “A railroad numbers its locomotives in order 1…N. One day you see a locomotive
with the number 60. Estimate how many locomotives the railroad has.”

In [ ]:
# names - max number of locomotives in the fleet
train = Pmf.getPmfFromSeq(1:1000 |> collect)

In [ ]:
"""
    Update Pmf (names are hypothesized max counts)

    data - observed counts
"""
function updateCounts!(pmf::Pmf.Pmf{Int}, data::Int)

    # the chance of seeing any number out of postulated N (max counts)
    # is 1/N
    likelihood::Vector{<:Float64} = 1 ./ pmf.names
    impossible::BitVector = data .> pmf.names
    likelihood[impossible] .= 0
    pmf.likelihoods .*= likelihood
    Pmf.updatePosteriors!(pmf, true)
    
    return nothing
end

In [ ]:
data = 60
updateCounts!(train, data)

In [ ]:
Pmf.drawLinesPosteriors(
    train,
    "Posterior distribution\n(after observing train 60)",
    "Number of trains",
    "PMF"
)

In [ ]:
Pmf.getNameMaxPosterior(train)

That might not seem like a very good guess; after all, what are the chances that
you just happened to see the train with the highest number? Nevertheless, if you
want to maximize the chance of getting the answer exactly right, you should
guess 60

An alternative is to compute the mean of the posterior distribution. Given a
set of possible quantities, $q_i$, and their probabilities, $p_i$, the mean of
the distribution is:

$mean = \sum_{i=1}^{n}p_{i}q_{i}$

In [ ]:
sum(train.posteriors .* train.names)

In [ ]:
Pmf.getMeanPosterior(train)

The mean of the posterior is 333, so that might be a good guess if you want to
minimize error. If you played this guessing game over and over, using the mean
of the posterior as your estimate would minimize the [mean squared
error](https://allendowney.github.io/ThinkBayes2/chap05.html) over the long run.

## Sensitivity to the Prior

In the previous section we started with the uniform prior from 1 to 1000.

In this case (little data) the posterior will depend on prior, e.g.
- upper bound: 500, posterior mean: 207.08
- upper bound: 1000, posterior mean: 333.42
- upper bound: 2000, posterior mean: 552.18

In [ ]:
upLimits  = [500, 1000, 2000]
for upLimit in upLimits
    testTrain = Pmf.getPmfFromSeq(1:upLimit |> collect)
    updateCounts!(testTrain, 60)
    println("$upLimit: $(Pmf.getMeanPosterior(testTrain))")
end

When the posterior is sensitive to the prior, there are tow ways to proceed:
- get more data
- get more background information and choose a better prior

So, if we started with the same uniform prior and observed trains number 60, 30, and 90.
The posterior would be:
- upper bound: 500, posterior mean: 151.85
- upper bound: 1000, posterior mean: 164.31
- upper bound: 2000, posterior mean: 171.34

The differences are smaller, but not enough for the posteriors to converge.

In [ ]:
upLimits  = [500, 1000, 2000]
cartNums = [30, 60, 90]
for upLimit in upLimits
    testTrain = Pmf.getPmfFromSeq(1:upLimit |> collect)
    foreach(x -> updateCounts!(testTrain, x), cartNums)
    println("$upLimit: $(Pmf.getMeanPosterior(testTrain))")
end

## Power Law Prior

If more data are not available, another option is to improve the priors by
gathering more background information.

In most fields, there are many small companies, fewer medium-sized companies,
and only one or two very large companies.

In fact, the distribution of company sizes tends to follow a power law, as
Robert Axtell reports in
[*Science*](https://www.science.org/doi/10.1126/science.1062081).

This law suggests that if there are 1000 companies with fewer than 10
locomotives, there might be 100 companies with 100 locomotives, 10 companies
with 1000, and possibly one company with 10,000 locomotives.

Mathematically, a power law means that the number of companies with a given
size, N, is proportional to $(1/N)^\alpha$, where $\alpha$ is a parameter that
is often near 1.

In [ ]:
alpha = 1.0
# a^(-n) = 1/(a^n)
train2 = Pmf.Pmf(train.names |> copy, train.names .^ (-alpha))
train2.priors ./= sum(train2.priors);

In [ ]:
Pmf.drawLinesPriors(
    train,
    "Prior distributions (uniform priors)",
    "Number of trains",
    "PMF"
)

In [ ]:
Pmf.drawLinesPriors(
    train2,
    "Prior distributions (power law priors)",
    "Number of trains",
    "PMF"
)

In [ ]:
updateCounts!(train2, 60)

In [ ]:
Pmf.drawLinesPosteriors(
    train,
    "Posterior distributions (uniform priors)",
    "Number of trains",
    "PMF"
)

In [ ]:
Pmf.drawLinesPosteriors(
    train2,
    "Posterior distributions (power law priors)",
    "Number of trains",
    "PMF"
)

In [ ]:
Pmf.getNameMaxPosterior(train2)

In [ ]:
Pmf.getMeanPosterior(train2)

The power law gives less prior probability to high values, which yields lower
posterior means, and less sensitivity to the upper bound.

Here’s how the posterior means depend on the upper bound when we use a power law
prior and observe three trains (for dataset: [30, 60, 90]):
- upper bound 500, posterior mean: 130.71
- upper bound 1000, posterior mean: 133.28
- upper bound 2000, posterior mean: 133.997

So the power law prior is more realistic, because it is based on general
information about the size of companies, and it behaves better in practice.

In [ ]:
upLimits  = [500, 1000, 2000]
cartNums = [30, 60, 90]
testTrain = 0
for upLimit in upLimits
    testTrain = Pmf.Pmf(1:upLimit |> collect, collect(1:upLimit) .^ (-alpha))
    testTrain.priors ./= sum(testTrain.priors)
    foreach(x -> updateCounts!(testTrain, x), cartNums)
    println("$upLimit: $(Pmf.getMeanPosterior(testTrain))")
end

## Credible Intervals

The `Pmf.getNameMaxPosterior` and `Pmf.getMeanPosterior` are both point
estimates of the quantity we are interested in.

Another way to summarize distribution is with percentiles.

Given `x` we can compute its **percentile rank** by finding all values <= `x`
and adding their probabilities.

In [ ]:
Pmf.getPosteriorsProbLEQ(testTrain, 100)

In [ ]:
"""
    Compute a quantile with the given prob.
"""
function getQuantile(pmf::Pmf.Pmf{T}, prob::Float64)::Float64 where {T<:Union{Int, Float64}}
    @assert (0 <= prob <= 1) "prob must be in range [0-1]"
    totalProb::Float64 = 0
    for (q, p) in zip(pmf.names, pmf.posteriors)
        totalProb += p
        if totalProb >= prob
            return q
        end
    end
    return -99.0
end

In [ ]:
getQuantile(testTrain, 0.5)

Now we can use it to get a 90% **credible interval**.

In [ ]:
[getQuantile(testTrain, q) for q in [0.05, 0.95]]

In [ ]:
function getCredibleInterval(pmf::Pmf.Pmf{T}, ci::Float64)::Vector{T} where {T<:Union{Int, Float64}}
    @assert (0.5 <= ci <= 0.99) "ci must be in range [0.5 - 0.99]"
    halfCI::Float64 = ci / 2
    return [getQuantile(pmf, q) for q in [0.5 - halfCI, 0.5 + halfCI]]
end

In [ ]:
getCredibleInterval(testTrain, 0.9)

## The German Tank Problem

During World War II a similar technique to the one used by us (`updateCounts`)
was used to estimated the production of german tanks.

For more on this problem, see [this Wikipedia page](https://en.wikipedia.org/wiki/German_tank_problem) and Ruggles and Brodie,
[available here](https://web.archive.org/web/20170123132042/https://www.cia.gov/library/readingroom/docs/CIA-RDP79R01001A001300010013-3.pdf).

## Informative Priors

Two approaches to chose priors:
- informative, based on background knowledge (downside: the choice is arbitrary)
- unifnformative, as unrestricted as possible (some say more objective)

With enough data the posteriors should converge anyway. Unfortunately with little data they will not.

The author (A.B. Downey) recomemnds to use the first approach.


## Exercises

### Exercise 1

Suppose you are giving a talk in a large lecture hall and the fire marshal
interrupts because they think the audience exceeds 1200 people, which is the
safe capacity of the room.

You think there are fewer then 1200 people, and you offer to prove it. It would
take too long to count, so you try an experiment:
- You ask how many people were born on May 11 and two people raise their hands.
- You ask how many were born on May 23 and 1 person raises their hand.
- Finally, you ask how many were born on August 1, and no one raises their hand.

How many people are in the audience? What is the probability that there are more
than 1200 people.

*Hint*: Remember the binomial distribution.


In [ ]:
# uniform priors for counts from 200 to 2400 (below and above would be easy to visually discern it from 1200)
ex1 = Pmf.getPmfFromSeq(200:2400 |> collect)
# ex1.names - number of people in the lecture hall
# ex1.priors - P(H)

In [ ]:
# now I need P(D|H) - likeliehood
# let's say a year got 365 days (we forget about leap years),
# so probability that a person was born in a given day is 1/365
# so P(D|H), that e.g. two people in the room are born on a given day can be
# calculated pretty fast with Distributions package

# 2 people born on May 11 for each postulated number of people in the lecture room
twoPeoplMay11 = Dsts.pdf.(Dsts.Binomial.(ex1.names, 1/365), 2);
# 1 pearson born on May 23 for each postulated number of people in the lecture room
onePersMay25 = Dsts.pdf.(Dsts.Binomial.(ex1.names, 1/365), 1);
# 0 people born on August 1 for each postulated number of people in the lecture room
zeroPeoplAug1 = Dsts.pdf.(Dsts.Binomial.(ex1.names, 1/365), 0);

In [ ]:
# since i got 2 people May 11 AND 1 person May 25 AND zero people Aug 1
# then I can use probability multiplication
# (assuming that the events are independent one from the other)
ex1.likelihoods = twoPeoplMay11 .* onePersMay25 .* zeroPeoplAug1;

In [ ]:
# and now for the posteriors
Pmf.updatePosteriors!(ex1, true);

In [ ]:
Pmf.drawLinesPosteriors(
    ex1,
    "Lecture room problem",
    "Number of people in the lecture room",
    "PMF"
    )

In [ ]:
# probability that there are more than 1200 people in the room
1 - Pmf.getPosteriorsProbLEQ(ex1, 1199)

In [ ]:
Pmf.getMeanPosterior(ex1)

In [ ]:
getCredibleInterval(ex1, 0.95)

### Exercise 2

I often see rabbits in the garden behind my house, but it’s not easy to tell
them apart, so I don’t really know how many there are.

Suppose I deploy a motion-sensing camera trap that takes a picture of the first
rabbit it sees each day. After three days, I compare the pictures and conclude
that two of them are the same rabbit and the other is different.

How many rabbits visit my garden?

To answer this question, we have to think about the prior distribution and the
likelihood of the data:
- I have sometimes seen four rabbits at the same time, so I know there are at
least that many. I would be surprised if there were more than 10. So, at least
as a starting place, I think a uniform prior from 4 to 10 is reasonable.
- To keep things simple, let’s assume that all rabbits who visit my garden are
equally likely to be caught by the camera trap in a given day. Let’s also assume
it is guaranteed that the camera trap gets a picture every day.

In [ ]:
ex2 = Pmf.Pmf(4:10 |> collect, 1 ./ collect(4:10))

In [ ]:
# so what is P(D|H)

# I got two the same rabbits and one different
# so I got one of the sequences:
# AAB or ABA or BAA (twice the same, and one different rabbit)
# so for 4 rabbits and observation: AAB, I got:
# 1/4 (seeing rabbit A) and 1/4 (seeing rabbit A) and 3/4 (seeing other rabbit)
# so in general: P(s) * P(s) * (1-P(s)) = P(s)^2 * (1-P(s))
# the same probability is for other two options
# so: (P(s)^2 * (1-P(s)))*3

function getProbAABrabbits(probRabbitA::Float64)::Float64
    @assert (0 <= probRabbitA <= 1) "probability must be in range [0-1]"
    return 3 * ((probRabbitA ^ 2) * (1 - probRabbitA))
end

ex2.likelihoods = getProbAABrabbits.(ex2.priors);

In [ ]:
Pmf.updatePosteriors!(ex2, true);

In [ ]:
Pmf.drawLinesPosteriors(
    ex2,
    "Rabbits in the garden estimate based on a phototrap",
    "Number of rabbits",
    "PMF"
)

### Exercise 3

Suppose that in the criminal justice system, all prison sentences are either 1,
2, or 3 years, with an equal number of each. One day, you visit a prison and
choose a prisoner at random. What is the probability that they are serving a
3-year sentence? What is the average remaining sentence of the prisoners you
observe?

In [ ]:
# names are sentences
# priors are probabilities of getting such a sentence at random
# while visiting a prison
ex3 = Pmf.Pmf(1:3 |> collect, 1:3 |> collect)

Priors should not be uniform (in the code snippet above they are not) if 1, 2, 3 years sententences are equally distributed.

Explanation: 
- we start with: 1, 2, 3, (1, 2, 3 years remained),
- after 1 yr we release the old 1yr and got: 2 (1 yr remained), 3 (2 years remained) + newly sentenced: 1, 2, 3 (1, 2, 3 years remained),
- after another yr we reduce the time left and again add newly sentenced.

So at any moment there should be more 3years than 2 years and 1 years (3 > 2 > 1).

So, P(3 years sentence) = 0.5

The average remaining sentence is:
- 3yr, 2yr, 1yr (for 3 year sentence)
- 2yr, 1yr (for 2 year sentence)
- 1yr (for 1 year sentence)

So: (3+2+1+2+1+1)/6 = 10/6 = $\frac{10}{6} = \frac{5}{3} = 1\frac{2}{3} = 1.66(7)$


Confirmation with a computer simulation

In [ ]:
mutable struct Prisoner
    sentenceYrs::Int
    remainedYrs::Int
end

In [ ]:
prisoners = [Prisoner(i, i) for i in 1:3]

In [ ]:
# a cycle is 3 years
function runNYears!(pris::Vector{<:Prisoner}, nYrs::Int = 3)
    for  _ in 1:nYrs
        foreach(p -> p.remainedYrs -= 1, pris)
        filter!(p -> p.remainedYrs != 0, pris)
        append!(pris, [Prisoner(i, i) for i in 1:3])
    end
end

In [ ]:
runNYears!(prisoners)

In [ ]:
prisoners

In [ ]:
length(filter(p -> p.sentenceYrs == 3, prisoners)) / length(prisoners)

In [ ]:
sum(map(p -> p.remainedYrs, prisoners)) / length(prisoners)